In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_excel('沪深300.xlsx', header=3)
df = df.dropna(axis = 1, how = 'all')

bool_cols = []
for column in df.columns:
    if df[column].dtype == 'bool':
        bool_cols.append(column)
        
df.drop(['tech_MA5', 'tech_MA20'], axis=1, inplace=True)

for column in df.columns:
    if df[column].isnull().any() and column not in bool_cols:
        df[column].fillna(df[column].mean(), inplace=True)
df['trend'] = (df['close'] < df['close'].shift(-1))
df = df.iloc[1:,]
df

,Date,open,high,low,close,volume,amt,dealnum,adjfactor,turn,...,stage_high,stage_low,up_days,down_days,breakout_ma,breakdown_ma,history_high_days,history_low_days,bull_bear_ma,trend
1,2012-05-29,2.602,2.661,2.602,2.644,714949008.0,1.875593e+09,11432,1.000000,6.936538,...,True,True,1.0,0.0,False,False,2,1,False,False
2,2012-05-30,2.642,2.647,2.633,2.636,265887198.0,7.017258e+08,5532,1.000000,2.595794,...,True,True,0.0,1.0,False,False,2,1,False,False
3,2012-05-31,2.621,2.645,2.613,2.630,178155984.0,4.681931e+08,4662,1.000000,1.747313,...,True,False,0.0,2.0,False,False,1,0,False,False
4,2012-06-01,2.629,2.660,2.620,2.630,179350035.0,4.734772e+08,5189,1.000000,1.766249,...,False,False,0.0,0.0,False,False,0,0,False,False
5,2012-06-04,2.601,2.604,2.555,2.558,546074272.0,1.413414e+09,7566,1.000000,5.559705,...,False,False,0.0,1.0,False,False,0,0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,2025-03-27,4.011,4.050,3.995,4.027,801525164.0,3.224973e+09,70576,1.207793,0.941116,...,False,False,1.0,0.0,False,False,0,0,False,False
3119,2025-03-28,4.027,4.029,4.000,4.008,704304791.0,2.826193e+09,59846,1.207793,0.828821,...,False,False,0.0,1.0,False,False,0,0,False,False
3120,2025-03-31,4.004,4.024,3.969,3.984,902172863.0,3.600807e+09,110122,1.207793,1.060626,...,False,False,0.0,2.0,False,False,0,0,False,False
3121,2025-04-01,3.988,4.003,3.979,3.982,615015541.0,2.452333e+09,58348,1.207793,0.721766,...,False,False,0.0,3.0,False,False,0,0,False,False


In [8]:
np.random.seed(42)
TEST_SIZE = 0.2
WINDOW_SIZE = 20

train_size = int(len(df) * (1 - TEST_SIZE))
test_data = df.loc[train_size+WINDOW_SIZE+1:, ['Date', 'close', 'trend']]
test_data

,Date,close,trend
2518,2022-09-29,3.894,False
2519,2022-09-30,3.868,False
2520,2022-10-10,3.789,False
2521,2022-10-11,3.786,True
2522,2022-10-12,3.848,False
...,...,...,...
3118,2025-03-27,4.027,False
3119,2025-03-28,4.008,False
3120,2025-03-31,3.984,False
3121,2025-04-01,3.982,False


In [9]:
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}


In [11]:
test_data['preds'] = np.random.choice([False, True], size=len(test_data))

#evaluate the model
print("\nFlipping coin model's performance:")
y_true = test_data['trend']
y_pred = test_data['preds']
metrics = calculate_metrics(y_true, y_pred)
for metric, value in metrics.items():
    print(f"{metric}: {value:.6f}")


Flipping coin model's performance:
Accuracy: 0.497521
Precision: 0.439446
Recall: 0.472119
F1 Score: 0.455197
